In [26]:
import requests
import pandas as pd
import os
from google.colab import userdata
from google import genai
from google.genai import types

# --- Funções ---

def baixar_planilha(url, nome_arquivo_local):
    """Baixa uma planilha de uma URL."""
    if os.path.exists(nome_arquivo_local):
        print(f"Arquivo já existe no diretório. Pulando o download.")
        return True
    else:
      try:
          response = requests.get(url)
          if response.status_code == 200:
              with open(nome_arquivo_local, 'wb') as f:
                  f.write(response.content)
              print(f"Planilha baixada com sucesso como '{nome_arquivo_local}'")
              return True
          else:
              print(f"Falha ao baixar a planilha. Código de status: {response.status_code}")
              return False
      except requests.exceptions.RequestException as e:
          print(f"Ocorreu um erro durante a requisição: {e}")
          return False

def ler_planilha_excel(nome_arquivo_local, header_row):
    """Lê uma planilha Excel e retorna um DataFrame."""
    if os.path.exists(nome_arquivo_local):
        try:
            df = pd.read_excel(nome_arquivo_local, header=header_row)
            print("Planilha lida com sucesso.")
            return df
        except Exception as e:
            print(f"Erro ao ler a planilha com pandas: {e}")
            return None
    else:
        print(f"Arquivo '{nome_arquivo_local}' não encontrado.")
        return None

def consultar_planilha_com_gemini(df, termo_busca, chat_session):
    """Consulta informações em um DataFrame e envia para o chat do Gemini."""


    coluna_substancia = 'SUBSTÂNCIA'
    coluna_tipo_produto = 'TIPO DE PRODUTO (STATUS DO PRODUTO)'
    coluna_produto = 'PRODUTO'
    coluna_apresentacao = 'APRESENTAÇÃO'
    coluna_valor_pmc_sem_impostos = 'PMC Sem Impostos'

    if df is not None:
        if coluna_tipo_produto not in df.columns:
          print(f"Coluna '{coluna_tipo_produto}' não encontrada na planilha.")
          return

        if coluna_substancia not in df.columns:
          print(f"Coluna '{coluna_substancia}' não encontrada na planilha.")
          return

        if coluna_produto not in df.columns:
          print(f"Coluna '{coluna_produto}' não encontrada na planilha.")
          return

        if coluna_substancia not in df.columns:
          print(f"Coluna '{coluna_substancia}' não encontrada na planilha.")
          return

        if coluna_apresentacao not in df.columns:
          print(f"Coluna '{coluna_apresentacao}' não encontrada na planilha.")
          return

        # Converter a coluna de produto para string para garantir que .str.contains funcione
        dados_filtrados = df[df[coluna_substancia].astype(str).str.contains(termo_busca, case=False, na=False)]

        if not dados_filtrados.empty:
            info_para_modelo = f"Considere as seguintes informações da planilha sobre {termo_busca}:\n"
            for index, row in dados_filtrados.iterrows():
                valor_substancia = row.get(coluna_substancia, 'N/A')
                valor_produto = row.get(coluna_produto, 'N/A')
                valor_apresentacao = row.get(coluna_apresentacao, 'N/A')
                valor_pmc_sem_impostos = row.get(coluna_valor_pmc_sem_impostos, 'N/A')
                valor_tipo_produto = row.get(coluna_tipo_produto, 'N/A')
                info_para_modelo += f"- Substância: {valor_substancia}, " \
                f"Valor PMC sem impostos: {valor_pmc_sem_impostos}," \
                f"Produto: {valor_produto}\n" \
                f"Tipo: {valor_tipo_produto}\n" \
                f"Apresentação: {valor_apresentacao}\n"

            pergunta = f"{info_para_modelo}\n" \
            f"Baseado nestas informações, gere uma tabela com os valores PMC sem impostos," \
            f"com nomes do Substância, produto, valores, tipo e Apresentação,"\
            f"utilize apenas produtos onde a substância é (única),"\
            f"não precisa informar que valores nan foram excluídos"
            f"{termo_busca}."

            resposta = chat_session.send_message(pergunta)
            print(resposta.text)
        else:
            print(f"{termo_busca} não encontrado na planilha ou nomes de coluna incorretos.")

    else:
        print("Não foi possível consultar informações pois o DataFrame não foi carregado.")

# --- Execução Principal ---

# Autenticação (mantenha no início, fora das funções se for global)
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

# Inicializar o cliente Gemini e a sessão de chat (pode ser feito aqui)
client = genai.Client()
modelo = "gemini-2.0-flash"
chat_config = types.GenerateContentConfig(system_instruction="")
chat = client.chats.create(model=modelo, config=chat_config)

# Definir URLs e nomes de arquivo
# Neste ponto entraria evolucoes para realizar a coleta dinamicamente dos novos arquivos liberados pela anvisa
url_da_planilha = 'https://www.gov.br/anvisa/pt-br/assuntos/medicamentos/cmed/precos/arquivos/xls_conformidade_site_20250509_101353847.xls/@@download/file'
nome_do_arquivo_local = 'xls_conformidade_site_20250509_101353847.xls'
header_linha = 41

# 1. Baixar a planilha
planilha_baixada = baixar_planilha(url_da_planilha, nome_do_arquivo_local)

# 2. Se a planilha foi baixada, ler o DataFrame
df_planilha = None
if planilha_baixada:
    df_planilha = ler_planilha_excel(nome_do_arquivo_local, header_linha)

# 3. Se o DataFrame foi lido, consultar com o Gemini
if df_planilha is not None:
    termo_busca = input("Digite a substância de busca (por exemplo, 'ACEBROFILINA'): ")

    consultar_planilha_com_gemini(
        df_planilha,
        termo_busca,
        chat
    )

Arquivo já existe no diretório. Pulando o download.
Planilha lida com sucesso.
Digite a substância de busca (por exemplo, 'ACEBROFILINA'): ACEBROFILINA
## Tabela de Preços de Acebrofilina (PMC sem Impostos)

| Substância     | Produto        | Tipo     | Apresentação                                                | Valor PMC sem Impostos (R$) |
|--------------|----------------|----------|-------------------------------------------------------------|-----------------------------|
| ACEBROFILINA | ACEBROFILINA   | Genérico | 5 MG/ML XPE CT FR VD AMB X 120 ML + COP                      | 25,63                       |
| ACEBROFILINA | ACEBROFILINA   | Genérico | 10 MG/ML XPE CT FR VD AMB X 120 ML + COP                     | 36,12                       |
| ACEBROFILINA | FILINAR G      | Novo     | 5 MG/ML GEL OR CT FR PLAS TRANS X 120ML + COL                | 25,63                       |
| ACEBROFILINA | FILINAR        | Similar  | 5 MG/ML XPE CT FR VD AMB X 120 ML + COP                  